In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import warnings
warnings.filterwarnings('ignore')

def preprocess_air_quality_data(file_path):
    # Load data
    df = pd.read_csv(file_path, low_memory=False)
    df.drop(columns=['City'], inplace=True)
    
    # Handle missing values
    numeric_columns = df.select_dtypes(include=['float64']).columns
    imputer = SimpleImputer(strategy='mean')
    df[numeric_columns] = imputer.fit_transform(df[numeric_columns])
    
    # Process datetime and create time features
    df['Datetime'] = pd.to_datetime(df['Datetime'], format='mixed', errors='coerce')
    df['Year'] = df['Datetime'].dt.year
    df['Month'] = df['Datetime'].dt.month
    df['Day'] = df['Datetime'].dt.day
    df['Hour'] = df['Datetime'].dt.hour
    
    # Categorize PM2.5 values
    bins = [0, 12, 35.4, 55.4, 150.4, 250.4, float('inf')]
    labels = ['Good', 'Moderate', 'Unhealthy for Sensitive', 'Unhealthy', 'Very Unhealthy', 'Hazardous']
    df['PM2.5_Category'] = pd.cut(df['PM2.5'], bins=bins, labels=labels)
    
    # Perform undersampling
    return df


# For each dataset
df = preprocess_air_quality_data('../Data/Aizawl_Amaravati_Amritsar_data.csv')

# Print sizes to see the reduction
print("Original sizes:", len(df))



df_tmp = df.copy()
df_tmp.drop(['AQI_Bucket', 'Datetime'], axis=1, inplace=True)

df_tmp['PM2.5_Category'] = df_tmp['PM2.5_Category'].astype("category").cat.as_ordered()

df_tmp['PM2.5_Category'].cat.categories

df_tmp['PM2.5_Category'].cat.codes
df_tmp.isnull().sum()/len(df_tmp)

np.random.seed(42)
from sklearn.model_selection import train_test_split

X = df_tmp.drop("PM2.5", axis=1)
Y = df_tmp["PM2.5"]

X_train, X_valid, Y_train, Y_valid = train_test_split(X, Y, train_size=0.8)

len(X_train), len(X_valid)

X_train.shape, X_valid.shape

pd.Categorical(X_train["PM2.5_Category"]).codes

for label, content in X_train.items():
    if not pd.api.types.is_numeric_dtype(content):
        #turn category into number
        X_train[label] = pd.Categorical(content).codes + 1


for label, content in X_valid.items():
    if not pd.api.types.is_numeric_dtype(content):
        X_valid[label] = pd.Categorical(content).codes + 1

Original sizes: 54733


In [4]:
%%time
# Lets build a machine learning model
from sklearn.ensemble import RandomForestRegressor

np.random.seed(42)
from sklearn.metrics import mean_squared_log_error, mean_absolute_error, r2_score, mean_squared_error

def rmsle(y_test, y_preds):
    """
    calculate root mean square log error between prediction and true labels    
    """

    return np.sqrt(mean_squared_log_error(y_test, y_preds))

# create a function to evaluate model on a few different levels

def show_scores(model):
    train_preds = model.predict(X_train)
    val_preds = model.predict(X_valid)
    scores = {"Training MAE" : mean_absolute_error(Y_train,train_preds),
              "valid MAE" : mean_absolute_error(Y_valid, val_preds),
              "Training RMSE" : np.sqrt(mean_squared_error(Y_train,train_preds)),
              "valid RMSE" : np.sqrt(mean_squared_error(Y_valid, val_preds)),
              "Training R^2": r2_score(Y_train,train_preds),
              "valid R^2" : r2_score(Y_valid, val_preds)}
    
    return scores

model = RandomForestRegressor(n_jobs=-1, random_state=42)
model.fit(X_train, Y_train)

show_scores(model)

CPU times: total: 1min 18s
Wall time: 9.34 s


{'Training MAE': 1.45758985649462,
 'valid MAE': 4.035261189034165,
 'Training RMSE': 4.2025884517785,
 'valid RMSE': 12.875389431562775,
 'Training R^2': 0.9894985771585165,
 'valid R^2': 0.9134356836816235}

In [5]:
%%time

# most ideal parameter 

ideal_model = RandomForestRegressor(n_estimators=90,
                                    min_samples_leaf=18,
                                    min_samples_split=3,
                                    max_features=0.5,
                                    n_jobs=-1,
                                    max_samples=None,
                                    random_state=42)  # random state so that our reult is reproducable

# FIT THE MODEL

ideal_model.fit(X_train, Y_train)
show_scores(ideal_model)

CPU times: total: 17.9 s
Wall time: 2.03 s


{'Training MAE': 4.114194377823058,
 'valid MAE': 4.8578659674672044,
 'Training RMSE': 12.91677441555488,
 'valid RMSE': 17.298148786969325,
 'Training R^2': 0.9007975107133815,
 'valid R^2': 0.843750940252573}

In [6]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(X_train,Y_train)
y_pred = linreg.predict(X_valid)
from sklearn.model_selection import cross_val_score
print('Cross_val_score', cross_val_score(linreg, X_train, Y_train, cv=10, scoring="r2").mean())
print('r2_score', r2_score(Y_valid, y_pred))
print("valid MAE ", mean_absolute_error(Y_valid, y_pred))
print("valid RMSE", np.sqrt(mean_squared_error(Y_valid, y_pred)))

Cross_val_score 0.6259833727664021
r2_score 0.5675312512464165
valid MAE  10.130490855757513
valid RMSE 28.778508819234354
